In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import pandas as pd
import numpy as np
df = pd.read_csv("drive/data/bangla_comment_update.csv");

In [0]:
df.head(5)

,label,comment
0,positive,চিন্তার কোনো কারন নেই আর কয়েক বছর পর আমি তোমার...
1,positive,আশা করছি বিসিবি পরের টেস্ট এর জন্য তোমাকে ডাকব...
2,offensive,"টেস্ট থেকে অবসর নিন দাদা,নয়তো টেস্টে খেলিয়ে আব..."
3,offensive,"হালা ডান্ডিয়া,,,ডেডা মাদারি,,তোর এই সব পালতু প..."
4,positive,মা ফাতেমা যদি মুসলিম সমাজের মা হতে পারে..মা খা...


In [0]:
df.tail(5)

,label,comment
7345,NaN,Wow বারা খান
7346,NaN,আপু তুমার চেহারা দেখলেই মনে হয় এখনই তুমি বান্দ...
7347,NaN,অনেকদিনের ইচ্ছে ছিলো পেত্নি দেখবো! আজ ইচ্ছেটা ...
7348,NaN,পেটে কৃমি পোজ দিয়ে বারা খান কি বুঝাতে চায়? বান...
7349,NaN,এর জন্য ভাষাকে কি যেমনে মনে চায় সেই ভাবে কি Ed...


In [0]:
col = ['label', 'comment']
df = df[col]
df = df[pd.notnull(df['comment'])]
print(df.loc[[7345]])
#print(df.tail())


     label        comment
7345   NaN  Wow বারা খান 


In [0]:
from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#everything needs to be digitalized. Labels and Texts.

#labels digitalized here in Y axis
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(df['label'])


#Posts digitalized here in x-axis
countvec = CountVectorizer();
X_data_full_tfidf = countvec.fit_transform(df['comment'])
# tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1')
# tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1,2))
# X_data_full_tfidf = tfidf.fit_transform(df['post'])


X_train, X_test, y_train, y_test = train_test_split(X_data_full_tfidf, y, test_size=0.4 , random_state = 52)

In [0]:
#with a naive bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC


clf = MultinomialNB().fit(X_train, y_train)
print("naive bayes accuracy is")

#step 4: evaluate classifier

print(clf.score(X_test, y_test))

#with a support vector machine
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
print("Support Vector with Linear Kernel Accuracy is:")
print(clf.score(X_test, y_test))

naive bayes accuracy is
0.5295918367346939
Support Vector with Linear Kernel Accuracy is:
0.5295918367346939
